# Project 6: The 3630 Drone Racing Challenge!

![Drone Racing Track](https://static.wixstatic.com/media/4bd4ae_57ef3d6e921f476a8cef1cb6fcb62ae7~mv2.jpg/v1/fill/w_2303,h_467,al_c,q_85,enc_auto/4bd4ae_57ef3d6e921f476a8cef1cb6fcb62ae7~mv2.jpg)

<p align="right">
<i> Image credits: <a href="https://www.aerialsports.tv"> Aerial Sports </a> </i>
</p>

"*Faster, faster, until the thrill of speed overcomes the fear of death!*" -- Hunter S. Thompson

Ladies and gentlemen, boys and girls, children of all ages! Welcome to the 6th (and final!) project of CS 3630!! In this project, we'll be flying drones from one place to another and racing through hoops, using path planning with drone dynamics! 

Have you ever wondered [how drones fly](https://www.youtube.com/watch?v=gsfkGlSajHQ)? How do they hover in the same position in the sky? How do they rotate about their axes and move towards their intended location in the three-dimensional world? We hope you figure all of this as you reach towards the last cells of this notebook! 

If you think that Quidditch is the most enticing sport in the world, then you are terribly mistaken. The coolest sport in today's 21st century world is DRONE RACING and if you don't believe this, you should check out this video!

<center>

[![Drone Racing](https://img.youtube.com/vi/bZvNLuC12R0/0.jpg)](https://www.youtube.com/watch?v=7wFEYnRVjc0 "Drone Racing")

</center>

Really cool, right?! But flying with code is even cooler! By the end of the project, you'll also be able to fly just like this!! 

### Logistics and Tips
- Project 6 will be released on November 22, Tuesday and will be due on December 6, Tuesday at 23:59. The late due date is December 9, Friday at 23:59, and late submission will incur a linear penalty as outlined in the syllabus.
- You need to submit your code and report on gradescope. Please read the submission guidelines below. 
- START EARLY!! This may take some time. 
- Press `Shift`+`Enter` to execute the current code cell, instead of clicking the play button with your mouse. 
- Press `b` (b for below) to add a code cell below the current one. 
- Press `a` (a for above) to add a code cell above the current one. 
- If you want cute doggos or kittens to help you get through this project, click on the Settings icon at the top-right $\rightarrow$ Miscellaneous $\rightarrow$ Kitty/Corgi mode 

Hope you enjoy working on this project as much as we did creating it! Let's [learn to fly!](https://www.youtube.com/watch?v=y3FNW6S2XAM) ^_^ 

### Grading Rubric
- Code: 50 points
- Reflection: 50 points



### Submission Details

#### Deliverables:

- `project6.py` - Export the `.py` file from Google Colab (File $\rightarrow$ Download) and upload it to the Gradescope assignment "Project 6 - Code".
- `project6_report.pdf` - Convert the given [Google Slides report](https://docs.google.com/presentation/d/1bP4fkRkbXeYfmSkoUnWLVnn1VskC_iu2Dovu5H5Qt5k/edit?usp=sharing) to a PDF, then submit the file to "Project 6 - Report" on Gradescope. Make sure to assign the corresponding pages to each question. Failure to submit the report in PDF form will result in your submission not being graded.

Please honor the [honor code](https://osi.gatech.edu/content/honor-code). This is an individual assignment and everyone should submit their own files and answers.

### Setup 
You'll need to run these cells whenever you fire up collab. But don't worry, it shouldn't take much time! 

In [1]:
## installation
%pip install -q -U gtbook

## imports
from typing import List
import random # no, use numpy rng
import math
import time

import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import gtsam
import pandas as pd
from gtbook import drone
import unittest

rng = np.random.default_rng(12345)

     |████████████████████████████████| 21.7 MB 1.3 MB/s 


In [2]:
t1 = gtsam.Point3(0.1, 0.05, 0.01) # front-left
t2 = gtsam.Point3(0.1,-0.05, 0.01) # front-right
t3 = gtsam.Point3(-0.1, 0, 0.01) # back

F,L,U = np.eye(3)
bTc1 = gtsam.Pose3(gtsam.Rot3(-L,-U,F), t1)
bTc2 = gtsam.Pose3(gtsam.Rot3(-L,-U,F), t2)
bTc3 = gtsam.Pose3(gtsam.Rot3(L,-U,-F), t3)

E,N,U = np.eye(3)
ntb = gtsam.Point3(100, 300, 10)
nRb = gtsam.Rot3(-N,E,U) # flying south, right of drone facing east
nTb = gtsam.Pose3(nRb, ntb)
wP = gtsam.Point3(103,295,12) # since our drone is now looking at back , in order to be its front, we decrease by 5.
bP = nTb.transformTo(wP)
print(f"bP = {bP} in (F,L,U) body frame")
c1P = bTc1.transformTo(bP)
print(f"c1P = {c1P} in camera frame 1")
c2P = bTc2.transformTo(bP)
print(f"c2P = {c2P} in camera frame 2")
w, h, f = 640, 480, 300
u0, v0 = float(w/2), float(h/2)
u1, v1 = u0 + f * c1P[0]/c1P[2], v0 + f * c1P[1]/c1P[2]
print(f"u1,v1 = {np.round([u1,v1],1)} in image 1")
u2, v2 = u0 + f * c2P[0]/c2P[2], v0 + f * c2P[1]/c2P[2]
print(f"u2,v2 = {np.round([u2,v2],1)} in image 2")

bP = [5. 3. 2.] in (F,L,U) body frame
c1P = [-2.95 -1.99  4.9 ] in camera frame 1
c2P = [-3.05 -1.99  4.9 ] in camera frame 2
u1,v1 = [139.4 118.2] in image 1
u2,v2 = [133.3 118.2] in image 2


In [3]:
vmap = {}
for theta_deg in [40]:
  theta_rad = math.radians(theta_deg)
  vmap[theta_deg] = 15. * math.sqrt(math.tan(theta_rad))
  print(f"v({theta_deg}) ~ {(vmap[theta_deg])} m/s")

v(40) ~ 13.740357237527997 m/s


We import `helpers.py` here that'll help us visualize our trees and maps. `gdown` should work fine since it's just one file, but in case you run into any issues, you can also manually download the `helpers.py` file from Canvas and upload it manually to your notebooks.

In [4]:
!pip install --upgrade --no-cache-dir gdown &> /dev/null
!gdown 16zEgTQf33ShfHK5dL6Ro8yYS3VEyPuhG

import helpers

Downloading...
From: https://drive.google.com/uc?id=16zEgTQf33ShfHK5dL6Ro8yYS3VEyPuhG
To: /content/helpers.py
100% 12.9k/12.9k [00:00<00:00, 30.3MB/s]


# Part 1: Rapidly-exploring Random Trees (RRT)! In 3 Dimensions! 



- RRT again? Yes you read that right! But this time, we're taking it up a notch and venturing into three dimensions! 😉
- Given the coordinates of our starting location and the destination location, how does a drone fly from the start to the goal? One easy yet effective way is to execute the RRT algorithm for the drone! 
- In this section, we shall implement the RRT algorithm in three dimensions (3D), because well, we live in a 3 dimensional world. 
- The code is a little different from project 4, but nevertheless, do quickly breeze through your project 4 code if you need to recall the basics of the RRT algorithm.
- In the previous project, each node in our RRT tree was an (x, y) tuple. Now that we're in three dimensions, we require three coordinates (x, y, z) to represent the position of the drone. We shall use `gtsam.Point3` to represent this. 

#### The GT 3630 Drone Racing Track!
Let's first quickly load up the racing track. Take a look at those suspended HOOPS!!! In mid air!! We're gonna do RRT and fly through 'em! 

In [5]:
start_race = gtsam.Pose3(r=gtsam.Rot3.Yaw(math.radians(45)), t=gtsam.Point3(10, 12, 2))
helpers.drone_racing_path(helpers.get_hoops(), start_race, [])

### RRT Components

We now code up the helper functions that'll help us execute the RRT algorithm. Please read the descriptions carefully for each of them, the arguments to the function, and the return types.

You will implement
- `generate_random_point` - generates a random point in the 3D configuration space
- `distance_euclidean` - computes the Euclidean distance between two points
- `find_nearest_node` - finds the nearest node in the tree
- `steer_naive` - steers the drone toward the target

You can refer to [Section 5.5.6](https://www.roboticsbook.org/S55_diffdrive_planning.html#a-simple-rrt-implementation) in the textbook for some helpful insights, especially for TODO 3.

#### TODO 1 - Generate random 3D point

Generate a random point in the 3D configuration space (10x10x10).

This can be achieved by sampling from a uniform distribution over the configuration space.

You must ensure that there is at least a 20% chance the target node itself is returned. 

Hint: Use `rng` defined in setup for random number generation.

In [6]:
# TODO 1
def generate_random_point(target_node: gtsam.Point3) -> gtsam.Point3:
  '''
  Arguments:
   - target_node: gtsam.Point3 (the goal itself!)
  
  Returns:
   - random_node: gtsam.Point3 - a random configuration of (x, y, z) which should be within the (10x10x10) space
  '''

  random_node = None

  ######## Student code here ########

  rints = rng.integers(low=0, high=10)
  if rints <=2:
    return target_node
  
  rdx = rng.uniform(0, 10)
  rdy = rng.uniform(0, 10)
  rdz = rng.uniform(0, 10)

  random_node = gtsam.Point3(rdx, rdy, rdz)

  ######## End student code  ########

  return random_node

#### TODO 2 - Compute Euclidean distance

Compute the Euclidean distance between two 3D points.

Hint: You may want to take a look at `np.linalg.norm`.

In [7]:
a = gtsam.Point3(1,1,1)
b= gtsam.Point3(2,2,2)
aa = np.array(a)
bb =np.array(b)
zz = np.array([0,0])
xx = np.array([2,2])
c = np.linalg.norm(zz-xx, axis= 0)
print(c)

2.8284271247461903


In [8]:
# TODO 2
def distance_euclidean(point_a: gtsam.Point3, point_b: gtsam.Point3) -> float:
  '''
  Arguments: 
   - point_a: gtsam.Point3
   - point_b: gtsam.Point3
  
  Returns: 
   - distance_euclidean_m: float
  '''

  distance_euclidean_m = None

  ######## Student code here ########
  distance_euclidean_m = np.linalg.norm(point_a- point_b, axis= 0)

  ######## End student code  ########

  return distance_euclidean_m

- Once we've sampled a random node, we need to find a potential parent for that node. We can do this by finding that node in the tree which is closest to the newly sampled node. 
- A naive way of doing this (as in Project 4) is to loop through all the nodes in the tree, compute the distance and pick the node with the minimum distance. 
- However, for this function, you must use numpy vectorization and parallelize your implementation. Refer to the [textbook](https://www.roboticsbook.org/S55_diffdrive_planning.html#a-simple-rrt-implementation) for some clues on how to do this! 

#### TODO 3 - Find nearest node

Given the current RRT tree and the newly sampled node, find the node in the tree which is closest to the newly sampled node, as well as the index of that node.

This can be done naively by looping through each node and computing the distance, but you need to parallelize your computation!

Hint: Refer to the textbook and numpy docs on how to use vectorization.

In [9]:
# TODO 3
def find_nearest_node(rrt: List[gtsam.Point3], new_node: gtsam.Point3):
  '''
  Arguments: 
   - rrt: List[gtsam.Point3] (the current tree)
   - new_node: gtsam.Point3 (the newly sampled point)

  Returns:
   - nearest_node: gtsam.Point3 (the nearest node!)
   - nearest_node_index: int (the index of the nearest node) 
  '''
  
  nearest_node = None
  nearest_node_index = None
  ######## Student code here ########
  # min = np.linalg.norm(rrt[0] - new_node, axis = 0)
  # nearest_node_index = 0
  # nearest_node = rrt[0]
  # for i in range(1, len(rrt)):
  #   curr = np.linalg.norm(rrt[i] - new_node, axis = 0)
  #   if curr < min:
  #     nearest_node_index = i
  #     nearest_node = rrt[i]
  distances = np.linalg.norm(np.array(rrt) - new_node, axis=1)
  nearest_node_index = np.argmin(distances)
  nearest_node = rrt[nearest_node_index]
  ######## End student code  ########

  return nearest_node, nearest_node_index

- Once we've sampled a node and found the node closest to it in the RRT tree, we need to find a node which would take us **in the direction** of the sampled node. In the previous project, we called this the `step_to_node`, as we wished to step in that direction. 
- In this 3-dimensional scenario, we shall call this the `steer_node` as we wish to steer our drone in the direction of the newly sampled node.
- We adopt a simple and naive strategy: we turn towards the target, and drive some **fraction of the distance**. (Hint: refer to the textbook!) 

#### TODO 4 - Steer toward the target

Steer the drone toward the target node. 

You'll want to first compute the displacement between the parent node and the target node.

The `steer_node` can then be found by taking a step in the direction of the target with a step size that is a fraction of the total displacement.

In [10]:
# TODO 4
def steer_naive(parent_node: gtsam.Point3, target_node: gtsam.Point3, fraction = 0.2):
  '''
  Arguments:
   - parent_node: gtsam.Point3
   - target_node: gtsam.Point3
  
  Returns: 
   - steer_node: gtsam.Point3
  '''

  steer_node = None

  ######## Student code here ########
  
  steer_node = parent_node + (target_node - parent_node)*fraction

  ######## End student code  ########
  
  return steer_node

Unit Tests for TODOs 1-4:

In [11]:
class TestRRT(unittest.TestCase):
  def test_generate_random_point(self):
    for _ in range(5):
      node = generate_random_point(gtsam.Point3(4,5,6))
      assert 0 <= node[0] <= 10
      assert 0 <= node[1] <= 10
      assert 0 <= node[2] <= 10
  
  def test_distance_euclidean(self):
    pt1 = gtsam.Point3(2.70109492, 4.55796488, 2.93292049)
    pt2 = gtsam.Point3(4, 7, 2)
    
    self.assertAlmostEqual(distance_euclidean(pt1, pt2), 2.9190804346571446, 2)

  def test_find_nearest_node(self):
    pt1 = gtsam.Point3(1,2,3)
    pt2 = gtsam.Point3(0.90320894, 3.55218386, 3.71979848)
    pt3 = gtsam.Point3(1.52256715, 4.24174709, 3.37583879)
    pt4 = gtsam.Point3(1.56803165, 4.10257537, 2.795647)
    pt5 = gtsam.Point3(2.68087164, 3.63713802, 4.25464017)
    new_point = gtsam.Point3(3.74935314, 3.2575652 , 5.20840562)
    
    rrt = [pt1, pt2, pt3, pt4, pt5]

    answer, index = find_nearest_node(rrt, new_point)
    assert (answer == pt5).all()
    assert (index == 4)
  
  def test_steer_naive(self):
    pt1 = gtsam.Point3(3.80319106, 2.49123788, 2.60348781)
    pt2 = gtsam.Point3(3.81712339, 0.33173367, 0.51835128)
    answer = gtsam.Point3(3.80597753, 2.05933704, 2.1864605)
    steer_node = steer_naive(pt1, pt2)
    
    assert(np.allclose(answer, steer_node, atol=1e-2))

suite = unittest.TestSuite()
suite.addTest(TestRRT('test_generate_random_point'))
suite.addTest(TestRRT('test_distance_euclidean'))
suite.addTest(TestRRT('test_find_nearest_node'))
suite.addTest(TestRRT('test_steer_naive'))

unittest.TextTestRunner().run(suite)

....
----------------------------------------------------------------------
Ran 4 tests in 0.019s

OK


<unittest.runner.TextTestResult run=4 errors=0 failures=0>

### Putting it all together! 

Let's now use all the functions coded up above and write the RRT loop. Here's the outline of the algorithm for your reference: 

1. Start with the RRT tree containing the start node. Our aim is to grow this tree with every iteration of the loop. 
2. Sample out a random node in the configuration space. Make sure you return the target node with 20% probability. 
3. Find the node nearest to the newly sampled node in the current tree, and make sure you keep track of the parent node. For every node that we add to the tree, you also need to store the index of its parent. 
3. Find the "steer node" - a node in the direction of the sampled node, and add it to the RRT tree. No need to check for obstacles, we don't have any! 😃
4. Repeat steps 2 and 3 until the distance of the latest node in the tree and the target node is less than the threshold. As soon as this terminating condition is reached, you can return the tree and the parents list. 

This function is the generalized RRT function which we shall be utilizing multiple times in this project. **It uses other helper functions as arguments!! So make sure you use the exact names as given in the function header.** This is **not** limited to the functions we've coded above. 

Note: There is no unit test for TODO 5. Please use the visualizations to see if your algorithm works as expected!



#### TODO 5 - RRT

This function is the main RRT loop and executes the entire RRT algorithm.

Follow the steps outlined above. You should keep sampling nodes until the terminating condition is met.

Please use the same function names as given in the function definition.

In [12]:
# TODO 5
def run_rrt(start_node, target_node, generate_random_node, steer, distance, find_nearest_node, threshold):
  '''
  Arguments: 
   - start_node: the start node, it could be gtsam.Point3 or gtsam.Pose3. 
   - target_node: the destination node, it could be gtsam.Point3 or gtsam.Pose3.
   - generate_random_node: this function helps us randomly sample a node
   - steer: this function finds the steer node, which takes us closer to our destination 
   - distance: this function computes the distance between the two nodes in the tree 
   - find_nearest_node: this function finds the nearest node to the randomly sampled node in the tree
   - threshold: float, this is used for the terminating the algorithm

  Returns:
   - rrt: List[gtsam.Point3] or List[gtsam.Pose3], contains the entire tree
   - parent_index_list: List[int], contains the index of the parent for each node in the tree
  '''

  rrt = []
  parent_index_list = []
  max_iterations = 2000
  rrt.append(start_node)
  parent_index_list.append(-1)
  #start_node: 출발점
  #target_node: 도착점
  #generate_random_node 뭘까. 펑션이라는데?
  #steer = step to 같은 펑션
  #distance: 이것도 거리구하는 펑션..
  #find_nearest_node: 딱봐도 펑션..
  #threshold: 현재 위치와 target_node거리가 범위안에 도달하면 종료한다.
  for i in range(max_iterations): 
    ######## Student code here ########
    random_node = generate_random_node(target_node)
    nearest_node, index = find_nearest_node(rrt, random_node)
    new_node = steer(nearest_node, random_node)
    rrt.append(new_node)
    parent_index_list.append(index)
    
    if distance(new_node, target_node) < threshold:
      return rrt, parent_index_list
    ######## End student code  ########
  
  return rrt, parent_index_list
# start_rrt_3d = gtsam.Point3(4, 5, 6)
# target_rrt_3d = gtsam.Point3(7, 4, 9)
# rrt_3d, parents_rrt_3d = run_rrt(start_rrt_3d, target_rrt_3d, generate_random_point, steer_naive, distance_euclidean, find_nearest_node, threshold=0.1)
# print("Nodes in RRT: ", len(rrt_3d))

### Visualizing the RRT Tree and Path!
Alright, with this we've completed an implementation of the RRT algorithm in 3 dimensions! Let's see what our tree and path look like! 

In [51]:
start_rrt_3d = gtsam.Point3(4, 5, 6)
target_rrt_3d = gtsam.Point3(7, 4, 9)
rrt_3d, parents_rrt_3d = run_rrt(start_rrt_3d, target_rrt_3d, generate_random_point, steer_naive, distance_euclidean, find_nearest_node, threshold=0.1)
print("Nodes in RRT: ", len(rrt_3d))

Nodes in RRT:  54


In [52]:
helpers.visualize_tree(rrt_3d, parents_rrt_3d, start_rrt_3d, target_rrt_3d)

- Remember how we kept track of the parent of every node added to the RRT tree? We shall use this information to print out the path from the start node to the target node. 
- We backtrack through each node's parent, starting from the solution node until the root, and store them in a `path` list.

Here's a method to do this. We've already implemented it for you 😉

In [15]:
def get_rrt_path(rrt: List[gtsam.Pose3], parents: List[int]) -> List[gtsam.Pose3]:
  path = []
  i = len(rrt) - 1 
  path.append(rrt[i])

  while(parents[i] != -1):
    next = rrt[parents[i]]
    path.append(next)
    i = parents[i]
  
  path.reverse()
  return path

path_rrt_3d = get_rrt_path(rrt_3d, parents_rrt_3d)
print("Length of Path: ", len(path_rrt_3d))
helpers.visualize_path(path_rrt_3d, start_rrt_3d, target_rrt_3d)

Length of Path:  20


## Reflection Questions

1. Paste a screenshot of the visualization of the RRT tree in your report. 
2. Paste a screenshot of your implementation of the `run_rrt` function. 
3. Paste a screenshot of the visualization of the final path obtained in your report. 
4. If you look closely at the tree and the path, you'll notice that as we reach close to the goal (the green node), the nodes are extremely close to one another! We begin to take really small steps and we converge very slowly towards it. Can you explain why this is happening?

# Part 2: Drone Dynamics

Now that we have a taste for path finding in 3D, let's see how a drone actually flies before we start steering it.

The kinematics of quadrotors and multicopters are those of simple rigid 3D bodies. The kinematics equations are most useful for navigation and control when expressed in the navigation frame $\mathcal{N}$, which for MAV applications is almost universally assumed to be non-rotating and aligned with gravity. In this project, we use the **ENU (East - North - Up)** navigation frame. This means that the x-axis points towards the east, the y-axis points towards the north, and the z-axis points upwards.

We also define a body frame $\mathcal{B}$ as having its origin at the center of mass of the vehicle. Following convention in aerospace applications, we fix the x-axis as pointing to the front of the vehicle (not always the direction of travel), the y-axis as pointing to the left, and the z-axis pointing up, the so-called **FLU (Forward - Left - Up)** convention.

We then define, respectively,
 - the vehicle's position $r^{n}\doteq[x,y,z]^{T}$,
 - its linear velocity $v^n=\dot{r^{n}}\doteq[u,v,w]^{T}$,
 - the attitude $R^n_b\doteq[i^b,j^b,k^b]\in SO(3)$, a $3\times 3$ rotation matrix from $\mathcal{B}$ to $\mathcal{N}$

Above, the superscript $n$ and $b$ denote quantities expressed in the *navigation* and *body* frame, respectively.

As we're now using the dynamics of the drone, we also need to represent the orientation of the drone as a rotation matrix. As you may have already guessed, we have an elegant structure that meets our needs - `gtsam.Pose3`.

The attitude and position can be represented by `gtsam.Rot3` and `gtsam.Point3` objects respectively. And they can be combined together to represent the *pose* of the drone using a `gtsam.Pose3` object. 

For example, if the current position of the drone is at the origin of $\mathcal{N}$ and $\mathcal{B}$ aligns with $\mathcal{N}$, the attitude is an identity matrix and the position is $[0, 0, 0]$. Let's see how we can represent this in code.

In [16]:
position_vector = gtsam.Point3(0, 0, 0)
attitude_mat33 = gtsam.Rot3()
pose_3d = gtsam.Pose3(r = attitude_mat33, t = position_vector)

print(f"Position: {position_vector}")
print(f"Attitude: {attitude_mat33}")
print(f"Pose: {pose_3d}")

Position: [0. 0. 0.]
Attitude: R: [
	1, 0, 0;
	0, 1, 0;
	0, 0, 1
]

Pose: R: [
	1, 0, 0;
	0, 1, 0;
	0, 0, 1
]
t: 0 0 0



We're going to use the following test suite to check your implementation of the different methods in this part. Feel free to add more test cases in the suite for your own testing.

Unit Tests for TODOs 6-8:

In [17]:
class TestDroneDynamics(unittest.TestCase):
  def test_compute_attitude_from_ypr(self):
    yaw = math.radians(45)
    pitch = math.radians(30)
    roll = math.radians(60)

    expected_attitude = gtsam.Rot3(
        [0.612372, 0.612372, -0.5],
        [-0.0473672, 0.65974, 0.75],
        [0.789149, -0.435596, 0.433013]
    )
    actual_attitude = compute_attitude_from_ypr(yaw, pitch, roll)

    assert(actual_attitude.equals(expected_attitude, tol=1e-2))
  
  def test_compute_force(self):
    attitude = gtsam.Rot3(
        [0.612372, 0.612372, -0.5],
        [-0.0473672, 0.65974, 0.75],
        [0.789149, -0.435596, 0.433013]
    )
    thrust = 20.0

    expected_force = gtsam.Point3(15.78, -8.71, 8.66)
    actual_force = compute_force(attitude, thrust)

    assert(np.allclose(actual_force, expected_force, atol=1e-2))
  
  def test_compute_terminal_velocity(self):
    force = gtsam.Point3(15.78, -8.71, 8.66)

    expected_terminal_velocity = gtsam.Point3(19.27, -14.32, 14.27)
    actual_terminal_velocity = compute_terminal_velocity(force)

    assert(np.allclose(actual_terminal_velocity, expected_terminal_velocity, atol=1e-2))


## Yaw, Pitch, and Roll

The attitude of the drone can also be defined using the yaw, pitch and roll angles of the drone with respect to $\mathcal{N}$. Our first task now will be to compute the attitude of the drone, $R^n_b$, given the yaw, pitch and roll angles of the drone.

#### TODO 6 - Compute attitude

Use yaw, pitch, and roll angles to compute the attitude of the drone.

Hint: Use `help(gtsam.Rot3)` to see different constructors.

In [18]:
# help(gtsam.Rot3)

In [19]:
# TODO 6
def compute_attitude_from_ypr(yaw_rad: float, pitch_rad: float, roll_rad: float) -> gtsam.Rot3:
  '''
  Arguments:
   - yaw_rad: float (in radians)
   - pitch_rad: float (in radians)
   - roll_rad: float (in radians)

  Returns:
   - attitude_mat33: gtsam.Rot3
  '''

  attitude_mat33 = None

  ######## Student code here ########

  # print(yaw_rad, pitch_rad, roll_rad)
  attitude_mat33 = gtsam.Rot3().Ypr(yaw_rad, pitch_rad, roll_rad)
  ######## End student code  ########

  return attitude_mat33
# suite = unittest.TestSuite()
# suite.addTest(TestDroneDynamics('test_compute_attitude_from_ypr'))

# unittest.TextTestRunner().run(suite)

In [20]:
suite = unittest.TestSuite()
suite.addTest(TestDroneDynamics('test_compute_attitude_from_ypr'))

unittest.TextTestRunner().run(suite)

.
----------------------------------------------------------------------
Ran 1 test in 0.004s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

Let us take the [Skydio 2+](https://www.skydio.com/skydio-2-plus) as a prototypical and relevant example. Its mass, with battery, is $800g$, but let's just say $1kg$ to make the calculations easier. Its dimensions are $229mm \times 274mm$, but let's assume the motors are at $(\pm 0.10m, \pm 0.10m)$, which is not too far off.

## Flying the drone

The four rotors on the drone provide a thrust upwards (in the body frame) to fly the drone. In general, the force $F^b_z$ aligned with the body z-axis will be:
$$F^b_z = \sum_{i=1}^4 f_i$$ where $f_i$ is the force applied by each rotor.

Of course, when we *tilt* the quadrotor forwards, we will direct some of that thrust towards generating horizontal acceleration. To get a handle on this, we need to calculate the thrust in the *navigation* frame, but this is just a matter of multiplying with the attitude $R^n_b$:

$$F^n = R^n_b \begin{bmatrix}0 \\ 0 \\ F^b_z\end{bmatrix} = \hat{z}^n_bF^b_z$$

See [Section 7.2.2](https://www.roboticsbook.org/S72_drone_actions.html#hover) and [Section 7.2.3](https://www.roboticsbook.org/S72_drone_actions.html#forward-flight).

Our next task is to compute the force vector $F^n$, given attitude $R^n_b$ and upwards thrust in the body frame, $F^b_z$.

#### TODO 7 - Compute force

Compute the force vector in the navigation frame given attitude and thrust in the body frame.


In [21]:
# TODO 7
def compute_force(attitude_mat33: gtsam.Rot3, thrust_newton: float) -> gtsam.Point3:
  '''
  Arguments:
   - attitude_mat33: gtsam.Rot3, nRb for the drone
   - thrust_newton: float, the upwards thrust produced by the 4 rotors
  
  Returns:
   - force_vector_newton: gtsam.Point3, the resultant force vector
  '''

  force_vector_newton = None

  ######## Student code here ########
  atm3 = attitude_mat33.matrix()
  fbz = gtsam.Point3(0,0,thrust_newton)

  force_vector_newton = np.dot(atm3,fbz)
  ######## End student code  ########

  return force_vector_newton
suite = unittest.TestSuite()
suite.addTest(TestDroneDynamics('test_compute_force'))

unittest.TextTestRunner().run(suite)


.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [22]:
suite = unittest.TestSuite()
suite.addTest(TestDroneDynamics('test_compute_force'))

unittest.TextTestRunner().run(suite)

.
----------------------------------------------------------------------
Ran 1 test in 0.006s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

## Drag and Terminal Velocity

Constant forward thrust, as calculated above, does *not* mean that the drone will continue accelerating, because of *drag*. In fact, from the spec-sheet of the Skydio-2 we see that the top speed (in autonomous mode) is $36\ Mph$, which is about $16\ m/s$, and the theoretical top speed is probably more like $20\ m/s$. Drag force increases *quadratically* with velocity:

$$F_{drag} \propto v^2$$

$$F_{drag} = k_dv^2$$

where $k_d$ is the drag coefficient. Using the specs from Skydio-2, we compute this $k_d = 0.0425$.

This drag force has to exactly balance the maximum forward thrust at terminal velocity:

$$F^n = k_dv_{terminal}^2$$

$$v_{terminal}^2 = \frac{F^n}{k_d} \implies v_{terminal} = \sqrt{\frac{F^n}{k_d}}$$ 

See [Section 7.2.5](https://www.roboticsbook.org/S72_drone_actions.html#drag-and-maximum-velocity).

Our task now is to calculate the terminal velocity vector, $v_{terminal}$, given the force vector, $F^n$ and the drag coefficient, $k_d$.

#### TODO 8 - Compute terminal velocity

Use the force vector and drag coefficient to compute the terminal velocity of the drone.

In [23]:
# TODO 8
def compute_terminal_velocity(force_vector_newton: gtsam.Point3, kd: float = 0.0425) -> gtsam.Point3:
  '''
  Arguments:
   - force_vector_newton: gtsam.Point3, the force vector in the navigation frame
   - kd: float, drag coefficient

  Returns:
   - terminal_velocity_ms: gtsam.Point3, the maximum velocity vector
  '''

  terminal_velocity_ms = None

  ######## Student code here ########

  v2 = force_vector_newton/kd
  # print(v2)
  signs = [1 if v2[0]>=0 else -1, 1 if v2[1]>=0 else -1,1 if v2[2]>=0 else -1]
  terminal_velocity_ms = gtsam.Point3(signs[0]*np.sqrt(np.abs(v2[0])),signs[1]*np.sqrt(np.abs(v2[1])),signs[2]*np.sqrt(np.abs(v2[2])))
  # print("개사그마",terminal_velocity_ms)
  ######## End student code  ########

  return terminal_velocity_ms
# suite = unittest.TestSuite()
# suite.addTest(TestDroneDynamics('test_compute_terminal_velocity'))

# unittest.TextTestRunner().run(suite)


In [24]:
suite = unittest.TestSuite()
suite.addTest(TestDroneDynamics('test_compute_terminal_velocity'))

unittest.TextTestRunner().run(suite)

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

# Part 3: Steering with Terminal Velocity

Now that we know how the thrust applied by the rotors flies the drone with a certain velocity, we can use that to explore the RRT tree.

However, since we learned that the drones now have *poses* instead of just positions, we will need to update our random node generating function to return random poses. Our first task, in this section, is to return a random pose node with the same random position component as before, but a new random attitude component using a random yaw, pitch and roll $\in [-60^\circ, 60^\circ]$.



Again, we're going to use the following test suite to check your implementation of the different methods in this part. Feel free to add more test cases in the suite for your own testing.

Unit Tests for TODOs 9-11:

In [25]:
class TestSteeringWithTerminalVelocity(unittest.TestCase):
  def test_generate_random_pose(self):
    target_node = gtsam.Pose3(r = gtsam.Rot3.Yaw(math.radians(45)), t = gtsam.Point3(8, 5, 6))
    for _ in range(5):
      random_node = generate_random_pose(target_node)
      assert(np.all(np.greater_equal(random_node.translation(), gtsam.Point3(0, 0, 0))))
      assert(np.all(np.less_equal(random_node.translation(), gtsam.Point3(10, 10, 10))))
      assert(np.all(np.greater_equal(random_node.rotation().ypr(), gtsam.Point3(math.radians(-60), math.radians(-60), math.radians(-60)))))
      assert(np.all(np.less_equal(random_node.rotation().ypr(), gtsam.Point3(math.radians(60), math.radians(60), math.radians(60)))))
  

  def test_find_nearest_pose(self):
    rrt_tree = [gtsam.Pose3(
                    r=gtsam.Rot3([1, 0, 0],
                                 [0, 1, 0],
                                 [0, 0, 1]),
                    t=gtsam.Point3(1, 2, 3)),
                gtsam.Pose3(
                    r=gtsam.Rot3([0.771517, -0.617213, 0],
                                 [0.0952381, 0.119048, -0.97619],
                                 [0.617213, 0.771517, 0.154303]), 
                    t=gtsam.Point3(2.70427, 3.90543, 3.85213)),
                gtsam.Pose3(
                    r=gtsam.Rot3([0.601649, -0.541882, 0.302815],
                                 [-0.301782, -0.62385, -0.516772],
                                 [0.627501, 0.29376, -0.721074]), 
                    t=gtsam.Point3(4.42268, 5.08119, 2.01005)),
                gtsam.Pose3(
                    r=gtsam.Rot3([-0.696943, 0.589581, -0.36631],
                                 [-0.664345, -0.416218, 0.594076],
                                 [0.204431, 0.679463, 0.704654]), 
                    t=gtsam.Point3(5.40351, 6.86933, 3.83104)),
                gtsam.Pose3(
                    r=gtsam.Rot3([-0.0686996, 0.218721, -0.818805],
                                 [-0.796488, -0.297401, -0.0126152],
                                 [-0.340626, 0.900832, 0.269211]), 
                    t=gtsam.Point3(1.43819, 5.96437, 4.97769))]
    new_node = gtsam.Pose3(
                    r=gtsam.Rot3([0.682707, 0.661423, 0.310534],
                                 [-0.626039, 0.748636, -0.218217],
                                 [-0.376811, -0.0454286, 0.925176]), 
                    t=gtsam.Point3(5.65333, 5.65964, 1.60624))
    expected_nearest_node = rrt_tree[2]
    expected_index = 2
    actual_nearest_node, actual_index = find_nearest_pose(rrt_tree, new_node)
    assert(actual_nearest_node.equals(expected_nearest_node, tol=1e-1))
    assert(actual_index == expected_index)

  
  def test_steer_with_terminal_velocity(self):
    current_node = gtsam.Pose3(gtsam.Rot3.Yaw(math.radians(90)), gtsam.Point3(1, 2, 3))
    new_node1 = gtsam.Pose3(gtsam.Rot3.Pitch(math.radians(45)), gtsam.Point3(8, 5, 6))
    new_node2 = gtsam.Pose3(gtsam.Rot3.Ypr(math.radians(45), math.radians(90), math.radians(0)), gtsam.Point3(5, 3, 4))

    expected_steer_node = gtsam.Pose3(gtsam.Rot3(
        [0.393919, -0.919145, 0],
        [0.336874, 0.144375, -0.930415],
        [0.855186, 0.366508, 0.366508]
    ), gtsam.Point3(3.00, 3.31, 4.31))
    actual_steer_node = steer_with_terminal_velocity(current_node, new_node1)

    assert(actual_steer_node.equals(expected_steer_node, tol=1e-2))


#### TODO 9 - Generate random pose

Generate a random pose in the configuration space (10x10x10). 

The attitude can be randomly sampled via yaw, pitch, and roll angles between -60 to 60 degrees.

You must ensure that there is at least a 20% chance that the targest pose itself is returned.

Hints:
- Use `rng` for random number generation.
- Use the `compute_attitude_from_ypr` function you implemented earlier.

In [26]:
# TODO 9
def generate_random_pose(target_pose_node: gtsam.Pose3) -> gtsam.Pose3:
  '''
  Arguments:
   - target_pose_node: gtsam.Pose3, the target pose for the RRT
  
  Returns:
   - random_pose_node: gtsam.Pose3, random pose or target pose
  '''

  random_pose_node = None

  ######## Student code here ########
  
  rints = rng.integers(low=0, high=10)
  if rints < 2:
    return target_pose_node
  rot = compute_attitude_from_ypr(math.radians(60),math.radians(60),math.radians(60))
  rx, ry, rz = rng.uniform(0, 10), rng.uniform(0, 10),rng.uniform(0, 10)

  random_pose_node = gtsam.Pose3(rot, gtsam.Point3(rx,ry,rz))
  ######## End student code  ########

  return random_pose_node
suite = unittest.TestSuite()
suite.addTest(TestSteeringWithTerminalVelocity('test_generate_random_pose'))

unittest.TextTestRunner().run(suite)

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [27]:
suite = unittest.TestSuite()
suite.addTest(TestSteeringWithTerminalVelocity('test_generate_random_pose'))

unittest.TextTestRunner().run(suite)

.
----------------------------------------------------------------------
Ran 1 test in 0.005s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

Since, we are using poses now, we will also need to update the way we have been calculating the distances between two nodes. You do not need to implement this, you can use the distance function we have provided in `helpers` as `distance_between_poses()`. Run `help(helpers.distance_between_poses)` to know how to use this function.

In [28]:
help(helpers.distance_between_poses)

Help on function distance_between_poses in module helpers:

distance_between_poses(pose1: gtsam.gtsam.Pose3, pose2: gtsam.gtsam.Pose3) -> float
    Computes the distance between two gtsam.Pose3 nodes in the tree
    Helpful functions: gtsam.Pose3.logmap, np.linalg.norm
    
    Arguments:
     - pose1: gtsam.Pose3
     - pose2: gtasm.Pose3
    
    Returns:
     - distance: float



We will now implement the function to find the nearest node in the RRT tree again, using the new distance function. This one is harder to vectorize, so we can implement it in the naive way using a `for` loop.

#### TODO 10 - Find nearest pose

Find the node in the current RRT tree that is nearest to the newly sampled node.

You can implement it the naive way using a `for` loop.

In [29]:
# TODO 10
def find_nearest_pose(rrt: List[gtsam.Pose3], new_pose_node: gtsam.Pose3):
  '''  
  Arguments: 
   - rrt: List[gtsam.Pose3] (a list of nodes currently in the tree) 
   - new_pose_node: gtsam.Pose3 (the newly sampled node in pose configuration)
  
  Returns:
   - nearest_pose: gtsam.Pose3 (the node in the tree which is CLOSEST to the newly sampled node)
   - nearest_pose_index: int (the index of the closest node, so we can keep track of the parent)
  '''

  nearest_pose = None
  nearest_pose_index = None


  ######## Student code here ########
  
  nearest_pose = rrt[0]
  nearest_pose_index = 0
  min = helpers.distance_between_poses(rrt[0],new_pose_node)
  # print("시작 최소값은",helpers.distance_between_poses( rrt[0],new_pose_node))
  for i in range(1, len(rrt)):
    curr = rrt[i]
    currmin = helpers.distance_between_poses( rrt[i],new_pose_node)
    if currmin < min:
      nearest_pose = rrt[i]
      nearest_pose_index = i
      min = currmin
      
  # print("리턴할떄 최소값은 ", min, "그인덱스는",nearest_pose_index)
  ######## End student code  ########
  
  return nearest_pose, nearest_pose_index


In [30]:
suite = unittest.TestSuite()
suite.addTest(TestSteeringWithTerminalVelocity('test_find_nearest_pose'))

unittest.TextTestRunner().run(suite)

.
----------------------------------------------------------------------
Ran 1 test in 0.004s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

Finally, we can rewrite the `steer()` function used in the RRT algorithm. We want to fly the drone with terminal velocity in the direction of the new random node for a specific duration, and return the node we reach.

In this part, we assume that we can apply the thrust in the direction we want to steer in, and we always apply the maximum thrust of $20N$. You may want to compute the following:
 - direction of travel
 - new attitude of the drone using the `get_new_attitude()` method from `helpers`
 - force vector using `compute_force()`
 - terminal velocity using `compute_terminal_velocity()`
 - new position by applying the velocity for the given duration

In [31]:
help(helpers.get_new_attitude)
# help(gtsam.Pose3)

Help on function get_new_attitude in module helpers:

get_new_attitude(current: gtsam.gtsam.Pose3, direction: <function Point3 at 0x7f721182b700>)
    Returns the new attitude of the drone given a current pose and the direction of travel
    
    Arguments:
     - current: gtsam.Pose3, the current pose of the drone
     - direction: gtsam.Point3, the direction vector for travel
    
    Returns:
     - attitude: gtsam.Rot3, the new attitude of the drone



#### TODO 11 - Steer with terminal velocity

We need to find a short steering from the current pose toward the target pose. 

We fly the drone for a small duration in the direction of the target at the terminal velocity with maximum possible thrust of 20N.

In [32]:
# help(gtsam.Pose3)

In [33]:
# TODO 11
def steer_with_terminal_velocity(current_pose_node: gtsam.Pose3, target_pose_node: gtsam.Pose3, duration_s: float = 0.1) -> gtsam.Pose3:
  '''
  Arguments:
   - current_pose_node: gtsam.Pose3, the current pose of the drone
   - target_pose_node: gtsam.Pose3, the target pose of the drone
   - duration_s: float, the duration of the flight, default: 0.1s
  
  Returns:
   - steer_pose_node: gtsam.Pose3, the new node reached by the drone
  '''
  
  steer_pose_node = None

  ######## Student code here ########
  # direction of travel
  tp = target_pose_node.translation()
  cp = current_pose_node.translation()
  direction = (tp - cp)/np.linalg.norm(tp-cp)

  # print("방향성:",direction)
  # new attitude of the drone using the get_new_attitude() method from helpers
  na = helpers.get_new_attitude(current_pose_node, direction)
  # print("뉴에티", na)
  # force vector using compute_force()
  fv = compute_force(na, 20)
  # terminal velocity using compute_terminal_velocity()
  tv = compute_terminal_velocity(fv)
  # print("캘리스프",tv)
  # new position by applying the velocity for the given duration
  steer_pose_node = gtsam.Pose3(na, cp + gtsam.Point3(tv*duration_s))


  ######## End student code  ########

  return steer_pose_node
suite = unittest.TestSuite()
suite.addTest(TestSteeringWithTerminalVelocity('test_steer_with_terminal_velocity'))

unittest.TextTestRunner().run(suite)

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [34]:
suite = unittest.TestSuite()
suite.addTest(TestSteeringWithTerminalVelocity('test_steer_with_terminal_velocity'))

unittest.TextTestRunner().run(suite)

.
----------------------------------------------------------------------
Ran 1 test in 0.005s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

### Putting it back into RRT

Now that we have coded the different components using drone dynamics, let us run the RRT algorithm again with the new steer function! 

In [35]:
start_rrt_drone = gtsam.Pose3(gtsam.Rot3(), gtsam.Point3(3, 6, 5))
target_rrt_drone = gtsam.Pose3(gtsam.Rot3(), gtsam.Point3(4, 8, 9))
rrt_drone, parents_rrt_drone = run_rrt(start_rrt_drone, target_rrt_drone, generate_random_pose, steer_with_terminal_velocity, 
                                       helpers.distance_between_poses, find_nearest_pose, threshold=3.0)
print("Number of RRT Nodes: ", len(rrt_drone))

Number of RRT Nodes:  8


In [36]:
helpers.visualize_tree(rrt_drone, parents_rrt_drone, start_rrt_drone, target_rrt_drone)

In [37]:
path_rrt_drone = get_rrt_path(rrt_drone, parents_rrt_drone)
print("Length of Path: ", len(path_rrt_drone))
print("The path obtained by steering with a terminal velocity:")
helpers.animate_drone_path(path_rrt_drone, start_rrt_drone, target_rrt_drone)

Length of Path:  3
The path obtained by steering with a terminal velocity:


### Reflection Questions

1. Paste a screenshot of your RRT tree visualization obtained after steering with a terminal velocity. 
2. Paste a screenshot of your final path obtained, after steering with a terminal velocity.
3. The `steer` function we coded up does not seem realistic in nature. What do you think are the two major limitations of our approach?  

# Part 4: Steering with more realistic drone dynamics

Two major limitations stand out in our previous approach of steering the drone:
1. We ignored the gravitational force affecting the direction and magnitude of our applied thrust. If you didn't believe this was important, [let Sheldon and Leonard tell you otherwise](https://www.youtube.com/watch?v=EgXOVGjIeyI).
2. By assuming we could apply the thrust in the direction we want to steer in, we allowed arbitrary and instantaneous attitude changes at the time of steering.

We are going to address these concerns in this section.

Here's the test suite for this part. Feel free to add your test cases for further testing.

Unit Tests for TODOs 12-13:

In [38]:
class TestRealisticSteer(unittest.TestCase):
  def test_compute_force_with_gravity(self):
    attitude = gtsam.Rot3(
        [0.612372, 0.612372, -0.5],
        [-0.0473672, 0.65974, 0.75],
        [0.789149, -0.435596, 0.433013]
    )
    thrust = 20.0

    expected_force = gtsam.Point3(15.78, -8.71, -1.34)
    actual_force = compute_force_with_gravity(attitude, thrust)

    assert(np.allclose(actual_force, expected_force, atol=1e-2))


  def test_steer(self):
    current_node = gtsam.Pose3(gtsam.Rot3.Yaw(math.radians(90)), gtsam.Point3(1, 2, 3))
    new_node1 = gtsam.Pose3(gtsam.Rot3.Pitch(math.radians(45)), gtsam.Point3(8, 5, 6))
    new_node2 = gtsam.Pose3(gtsam.Rot3.Ypr(math.radians(45), math.radians(90), math.radians(0)), gtsam.Point3(5, 3, 4))

    expected_steer_node = gtsam.Pose3(gtsam.Rot3(
        [0.17, 0.97, -0.17],
        [-0.96, 0.20, 0.17],
        [ 0.20, 0.14, 0.97]
    ), gtsam.Point3(1.97, 2.81, 4.49))
    actual_steer_node = steer(current_node, new_node1)
    assert(actual_steer_node.equals(expected_steer_node, tol=1e-2))


## Hovering the drone

To hover, assuming $g=10\ m/s^2$, the four rotors have to provide a thrust of $10N$ upwards to compensate for gravity, i.e., $2.5N$ per motor. Of course, we need to be able to accelerate upwards, so let's assume each motor can provide up to double that, i.e., $0$ to $5N$. So, while the drone is level, here are some sample accelerations we can deliver (note that the drone has a mass of $1kg$):
 - $f_i=0N$ for $i\in{1..4}$: downwards acceleration at $-10\ m/s^2$
 - $f_i=2.5N$ for $i\in{1..4}$: stable hover $0\ m/s^2$
 - $f_i=5N$ for $i\in{1..4}$: upwards acceleration at $10\ m/s^2$

## Correcting force vector

We will now correct our implementation of `compute_force` to incorporate the effect of the gravitational force. We can assume $g = 10\ m/s^2$.

#### TODO 12 - Compute force with gravity

Compute the net force vector given attitude and thrust in the body frame by adjusting for the downwards weight force.

In [39]:
# TODO 12
def compute_force_with_gravity(attitude_mat33: gtsam.Rot3, thrust_newton: float, mass_kg: float = 1.0) -> gtsam.Point3:
  '''
  Arguments:
   - attitude_mat33: gtsam.Rot3, nRb for the drone
   - thrust_newton: float, the upwards thrust produced by the 4 rotors
   - mass_kg: float, the mass of the drone, default: 1.0kg
  
  Returns:
   - force_newton: gtsam.Point3, the resultant force vector
  '''

  force_newton = None

  ######## Student code here ########
  # print("입력값:", attitude_mat33, " 쓰뉴,", thrust_newton, " 메키", mass_kg)
  atm3 = attitude_mat33.matrix()
  fbz = gtsam.Point3(0,0,thrust_newton) #여기서 masskg안하고 밑에 10에다가 masskg를 곱ㅎㅏ는 방안도있다..
  # print("atm:", atm3, "fbz:", fbz)
  force_newton = np.dot(atm3, fbz) -gtsam.Point3(0,0,10 *mass_kg)
  # print("리턴값", force_newton)
  #  expected_force = gtsam.Point3(15.78, -8.71, -1.34) 

  ######## End student code  ########

  return force_newton
# suite = unittest.TestSuite()
# suite.addTest(TestRealisticSteer('test_compute_force_with_gravity'))

# unittest.TextTestRunner().run(suite)

In [40]:
suite = unittest.TestSuite()
suite.addTest(TestRealisticSteer('test_compute_force_with_gravity'))

unittest.TextTestRunner().run(suite)

.
----------------------------------------------------------------------
Ran 1 test in 0.005s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

## Reflection questions
Since we now have the correct drone dynamics in place, let's answer a few reflection questions.

Play with the configurations of yaw, pitch, roll, and thrust (in the cell below) and answer these questions for each configuration:
 1. In which direction is the drone flying in the navigation frame? (Hint: terminal velocity)
 2. Is the drone flying upwards, downwards or maintaining level flight? (Look at terminal velocity or net force)
 3. In what direction is the thrust applied in the navigation frame? (Think orientation along z-axis)
 4. What is the speed of the drone? (Hint: terminal velocity)
 5. What direction is the front of the drone facing in the navigation frame? (Think orientation along x-axis)

Feel free to add your own code to the cell below, if you need it.

**Configurations:** All angles are in degrees and thrust is in Newtons.

| Yaw &nbsp; &nbsp; &nbsp;  &nbsp; &nbsp; &nbsp; | Pitch &nbsp; &nbsp; &nbsp;  &nbsp; &nbsp; &nbsp; | Roll &nbsp; &nbsp; &nbsp;  &nbsp; &nbsp; &nbsp;| Thrust &nbsp; &nbsp; &nbsp;  &nbsp; &nbsp; &nbsp;|
|-----|-------|------|--------|
| 90  |   0   |  0   |  10    |
| 0   |  45   |  45  |  20    |
| 25  |  0    |  45  |  15    |
| 10  |  45   |  30  |  0     |


In [41]:
# Use for the reflection questions above

# Controllers
yaw_rad = math.radians(90)
pitch_rad = math.radians(0)
roll_rad = math.radians(0)
thrust_newton = 10

# Computing drone velocity
nRb_mat33 = compute_attitude_from_ypr(yaw_rad, pitch_rad, roll_rad)
net_force_newton = compute_force_with_gravity(nRb_mat33, thrust_newton)
terminal_velocity_ms = compute_terminal_velocity(net_force_newton)

print(f"Orientation in navigation frame (nRb):\n {np.round(nRb_mat33.matrix(), 2)}")
print(f"Net force (N):\n {np.round(net_force_newton, 2)}")
print(f"Terminal velocity (m/s):\n {np.round(terminal_velocity_ms, 2)}")

Orientation in navigation frame (nRb):
 [[ 0. -1.  0.]
 [ 1.  0.  0.]
 [-0.  0.  1.]]
Net force (N):
 [0. 0. 0.]
Terminal velocity (m/s):
 [0. 0. 0.]


## Correcting drone attitude while steering

In this section, we will implement a more realistic version of the `steer()` function. We cannot just rotate the drone into any direction of travel. We limit the yaw, pitch, and roll rotations of the drone to $[-10^\circ, 10^\circ]$ for a more realistic instantaneous change in attitude. We also attempt to find a thrust value that would take us closest to the new random node, for each yaw, pitch, and roll.

To make it more simple, we select a yaw from $3$ values: $[-10^∘, 0^∘, 10^∘]$, a pitch from $3$ values: $[-10^∘, 0^∘, 10^∘]$, a roll from $3$ values: $[-10^∘, 0^∘, 10^∘]$, and a thrust from $4$ values: $[5, 10, 15, 20]$. For each of the $108$ combinations, we find the node we can steer to by finding the new attitude and applying the terminal velocity computed using the updated force function for a specific duration. We return the node that is closest to the target node we are steering towards.

#### Hint for calculating the new attitude:

The yaw, pitch, and roll rotations of $[-10^∘, 0^∘, 10^∘]$ are in the body frame. You can use the existing attitude and this rotation in body frame to compute the new attitude.

$$R^n_{b_1} = R^n_{b_0} R^{b_0}_{b_1}$$

#### TODO 13 - More realistic steer

Steer the drone with limits on rotation change and thrust values using force computations with gravity.

In [42]:
# TODO 13
def steer(current_pose_node: gtsam.Pose3, target_pose_node: gtsam.Pose3, duration_s = 0.1):
  '''
  Arguments:
   - current_pose_node: gtsam.Pose3, the current pose of the drone
   - target_pose_node: gtsam.Pose3, the target pose of the drone
   - duration_s: float, the duration of the flight, default: 0.1
  
  Returns:
   - steer_pose_node: gtsam.Pose3, the new node reached by the drone
  '''

  steer_pose_node = None
  yaw_values_deg = [-10, 0, 10]
  pitch_values_deg = [-10, 0, 10]
  roll_values_deg = [-10, 0, 10]
  thrust_values_newton = [5, 10, 15, 20]

  ######## Student code here ########
  #   In this section, we will implement a more realistic version of the steer() function. 
  #   We cannot just rotate the drone into any direction of travel. 
  #   We limit the yaw, pitch, and roll rotations of the drone to [−10∘,10∘]  회전 범위를 준게아니라 돌거면 -10, 10 으로 돌고 아닐거면 0 으로 하란 소린거같은데.
  #   for a more realistic instantaneous change in attitude. 
  #   We also attempt to find a thrust value that would take us closest to the new random node, for each yaw, pitch, and roll. 각각도는거에 대해 가장 가까운 곳으로 배달시켜주는 thrust찾아
  # To make it more simple, we select a yaw from 3 values: [−10∘,0∘,10∘], a pitch from 3 values: [−10∘,0∘,10∘], a roll from 3 values: [−10∘,0∘,10∘],
  #  and a thrust from 4 values: [5,10,15,20]. For each of the 108 combinations, we find the node we can steer to by finding the new attitude #다돌리면서 attr를 만들어요.
  #  and applying the terminal velocity computed using the updated force function for a specific duration.  #computeforce쓰나? 암튼 듀레이션박아서구한 tv를 적용함.
  #  We return the node that is closest to the target node we are steering towards. #그 중에서 나온값들로 가장 타겟위치와 가까운 노드를 리턴해주세요...
  # Hint for calculating the new attitude:
  # The yaw, pitch, and roll rotations of  [−10∘,0∘,10∘]  are in the body frame. You can use the existing attitude and this rotation in body frame to compute the new attitude.
  #에티튜드 = R붙은애들임..
  cp = current_pose_node.translation()
  prev = current_pose_node.rotation()
 # min = helpers.distance_between_poses(prev, target_pose_node)
  nodes = []
  dist = []
  for i in range(len(yaw_values_deg)):
    y = math.radians(yaw_values_deg[i])
    for j in range(len(pitch_values_deg)):
      p = math.radians(pitch_values_deg[j])
      for k in range(len(roll_values_deg)):
        r = math.radians(roll_values_deg[k])
        for h in range(len(thrust_values_newton)):
          t = thrust_values_newton[h]
          na = prev *compute_attitude_from_ypr(y,p,r)
          nf = compute_force_with_gravity(na, t)
          tv = compute_terminal_velocity(nf)
          node = gtsam.Pose3(na,cp + gtsam.Point3(tv*duration_s))
          nodes.append(node)
          dist.append(helpers.distance_between_poses(node, target_pose_node))
          # print("y:",y,"p:",p,"r:",r,"t:",t,"\n노드", node)

          # if dis < min:
          #   print(dis , "vs", min, "\n", node)
          #   steer_pose_node = node
          #   min = dis
  steer_pose_node = nodes[dist.index(min(dist))]
          
  ######## End student code  ########
  # expected_steer_node = gtsam.Pose3(gtsam.Rot3(
  #       [0.17, 0.97, -0.17],
  #       [-0.96, 0.20, 0.17],
  #       [ 0.20, 0.14, 0.97]
  #   ), gtsam.Point3(1.97, 2.81, 4.49))
  return steer_pose_node
# suite = unittest.TestSuite()
# suite.addTest(TestRealisticSteer('test_steer'))

# unittest.TextTestRunner().run(suite)


In [43]:
suite = unittest.TestSuite()
suite.addTest(TestRealisticSteer('test_steer'))

unittest.TextTestRunner().run(suite)

.
----------------------------------------------------------------------
Ran 1 test in 0.008s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

## RRT with drone dynamics! 

Now that we have coded the appropriate drone dynamics, let us plug the `steer()` method back into our RRT function

In [44]:
start_rrt_drone_realistic = gtsam.Pose3(gtsam.Rot3(), gtsam.Point3(2, 3, 4))
target_rrt_drone_realistic = gtsam.Pose3(gtsam.Rot3(), gtsam.Point3(7, 4, 6))
rrt_drone_realistic, parents_rrt_drone_realistic = run_rrt(start_rrt_drone_realistic, target_rrt_drone_realistic, 
                                                           generate_random_pose, steer, helpers.distance_between_poses, 
                                                           find_nearest_pose, threshold=1.5)
print("Nodes in RRT Tree: ", len(rrt_drone_realistic))

Nodes in RRT Tree:  35


Let us see how our new RRT algorithm now performs with the same start and target poses.

In [45]:
helpers.visualize_tree(rrt_drone_realistic, parents_rrt_drone_realistic, start_rrt_drone_realistic, target_rrt_drone_realistic)

In [46]:
path_rrt_drone_realistic = get_rrt_path(rrt_drone_realistic, parents_rrt_drone_realistic)
print("Length of Path: ", len(path_rrt_drone_realistic))
helpers.animate_drone_path(path_rrt_drone_realistic, start_rrt_drone_realistic, target_rrt_drone_realistic)

Length of Path:  8


## Reflection Questions

1. Paste a screenshot of your RRT tree visualization obtained after implementing a more realistic version of the `steer` function. 
2. Paste a screenshot of your final path obtained after implementing a more realistic version of the `steer` function. 
3. Open-ended: Do you think the drone can fly faster than our current implementation using these realistic dynamics? If yes, how? If no, why?

# Part 5: Drone Racing! 

Alright folks, now that we've successfully implemented the RRT algorithm whilst incorporating drone dynamics, it is now time to take your drones to the **GT 3630 Drone Racing Challenge**!! ARE YOU READY?!?! Check out this exhilarating racing video to get your adrenaline rushing!   

<center>

[![Drone Racing](https://img.youtube.com/vi/dDTqHYbpCVw/0.jpg)](https://www.youtube.com/watch?v=dDTqHYbpCVw "Drone Race")

</center>



- The idea is simple; there are 4 hoops in the GT 3630 Racing Track, which are given to you as `target_poses`. Your objective is to fly through those hoops in the **same order** and reach the final treasure, which is present at the center of the last hoop in the list. 
- We know that RRT plans a path from one start point to one destination point. Since we have multiple hoops to fly through, we can consider them as **intermediate goal points**. 
- So you can first use the RRT algorithm to plan a path from the start location to the first hoop (an intermediate goal), then use RRT again to plan your path from that point to the next hoop, and so on and so forth! 
- As you would've noticed in Part 4, the path generated by RRT gets us really close to the goal, but doesn't get us exactly there. So we've provided you with a magical function `pass_through_the_hoop`, which performs some wizardry and gets your drone through the hoop 😉 

You need to perform RRT with a set of intermediate goal points, use `pass_through_the_hoop` to successfully traverse through the hoops, and obtain your final treasure! 

When you call RRT with `run_rrt`, make sure you pass the correct functions as arguments! **You can play around with the threshold, but do NOT exceed a  threshold value of 3**


#### TODO 14 - Race with RRT

Run RRT multiple times to travel through the hoops in the order given by `target_poses`.

Note: Since the drone can only approximately reach the target nodes with our RRT implementation, we add multiple points at the end of each sub-path to pass through the hoop. Use `helpers.pass_through_the_hoop(target_pose, path)` after calculating each sub-path to append these points to the path.

In [47]:
help(helpers.pass_through_the_hoop)

Help on function pass_through_the_hoop in module helpers:

pass_through_the_hoop(target: gtsam.gtsam.Pose3, path: list)
    This function updates the path generated by RRT, thereby helping 
    the drone to pass through the hoop, after getting close enough 
    
    Arguments: 
     - target: gtsam.Pose3, the hoop through which we need to pass
     - path: the current path of the drone generated by RRT



In [62]:
# TODO 14
def drone_racing_rrt(start_pose: gtsam.Pose3, target_poses: List[gtsam.Pose3]) -> List[gtsam.Pose3]:
  '''
  Arguments:
   - start_pose: gtsam.Pose3, initial pose of the drone
   - target_poses: List[gtsam.Pose3], list of hoop poses (RRT targets)

  Returns:
   - drone_path: List[gtsam.Pose3], entire path from start to last hoop
  '''

  drone_path = []
  # fam = {} #가족관계증명서
  ######## Student code here ########
  threshold = 3
  curr = start_pose
  # subtree = [curr]
  # targets = [start_pose]
  # drone_path.append(curr)
  for i in range(len(target_poses)):
    if i > 0:
      start_pose = path[len(path)-1]
    rrt, rrtp = run_rrt(start_pose, target_poses[i], generate_random_pose, steer, helpers.distance_between_poses,find_nearest_pose, threshold)
    path = get_rrt_path(rrt, rrtp)
    helpers.pass_through_the_hoop(target_poses[i],path)
    
   

    for k in range(len(path)):
      drone_path.append(path[k])    



    # goal = target_poses[i]
    # while helpers.distance_between_poses(curr, goal) > threshold:

  #     #rrt를 찾으러가자
  #     #골로 향하는 랜덤노드
  #     random = generate_random_pose(goal)
  #     #렌덤노드로 갈 수 있는 가장가까운 노드 
  #     near, _ = find_nearest_pose(subtree, random)
  #     #랜덤으로 갈 수 있는 가장가까운 지점에서 랜덤으로 점프
  #     new_node = steer(near, random)
  #     fam[new_node] = near
  #     subtree.append(new_node)
  #     curr = new_node
  #   if helpers.distance_between_poses(new_node, goal) <= threshold:
  #     #현재지점에서 이번ㄹ ㅜ프의 시작점으로 역행한뒤 그 내역을 drone_path에 넣으세요. 그러면 알짜베기만들어가겠죠?
  #     helpers.pass_through_the_hoop(goal, subtree)
  #     nxt = subtree[len(subtree)-1]
  #     fam[nxt] = curr

      
  #     curr = nxt
  #     subtree = [nxt]
  # stk = [nxt]
  # while fam.get(curr): #뭔가.. 젤 처음게 문젠거같아.. 
  #   stk.append(curr)
  #   curr = fam[curr]
  #   # print(curr)
  # # print(len(stk))
  # while stk:
  #   p = stk.pop()
  #   drone_path.append(p)

  ######## End student code  ########

  return drone_path
start_rrt_drone_race = gtsam.Pose3(r=gtsam.Rot3(), t=gtsam.Point3(10, 12, 2))
targets_rrt_drone_race = helpers.get_targets()
path_rrt_drone_race = drone_racing_rrt(start_rrt_drone_race, targets_rrt_drone_race)
helpers.drone_racing_path(helpers.get_hoops(), start_rrt_drone_race, path_rrt_drone_race)
    

In [49]:
start_rrt_drone_race = gtsam.Pose3(r=gtsam.Rot3(), t=gtsam.Point3(10, 12, 2))
targets_rrt_drone_race = helpers.get_targets()
path_rrt_drone_race = drone_racing_rrt(start_rrt_drone_race, targets_rrt_drone_race)

[R: [
	0.866025, 0.353553, 0.353553;
	0, 0.707107, -0.707107;
	-0.5, 0.612372, 0.612372
]
t: 4.35355 2.29289 12.6124
]


In [50]:
helpers.drone_racing_path(helpers.get_hoops(), start_rrt_drone_race, path_rrt_drone_race)

### Reflection Questions

1. Paste a screenshot of your drone's trajectory at the GT 3630 Drone Racing Challenge, passing through all the hoops in the circuit. 
2. Paste a screenshot of your implementation of the `drone_racing_rrt` function.  
3. Open-ended: What do you think can be some ways to optimize the trajectory taken by the drone? Keep in mind that instantaneous rotation of the drone is limited to -10 to 10 degrees in yaw, pitch and roll.

### And that's all! Congratulations on completing the GT 3630 Drone Racing Challenge! 🎉